In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
data = pd.read_csv('hh_database.csv', sep=';')
print(data.shape)

(44744, 12)


In [6]:
data.loc[data['Опыт работы']==0]

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто


In [4]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44744 entries, 0 to 44743
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Пол, возраст                     44744 non-null  object
 1   ЗП                               44744 non-null  object
 2   Ищет работу на должность:        44744 non-null  object
 3   Город, переезд, командировки     44744 non-null  object
 4   Занятость                        44744 non-null  object
 5   График                           44744 non-null  object
 6   Опыт работы                      44576 non-null  object
 7   Последнее/нынешнее место работы  44743 non-null  object
 8   Последняя/нынешняя должность     44742 non-null  object
 9   Образование и ВУЗ                44744 non-null  object
 10  Обновление резюме                44744 non-null  object
 11  Авто                             44744 non-null  object
dtypes: object(12)
memory usage: 4.1+

None

In [5]:
# Преобразование данных

# ВУЗ
def get_education(arg):
    arg = ' '.join(arg.split(' ')[:3])
    if 'Высшее' in arg:
        return 'высшее'
    elif 'Неоконченное высшее' in arg:
        return 'неоконченное высшее'
    elif 'Среднее специальное' in arg:
        return 'среднее специальное'
    elif 'Среднее образование' in arg:
        return 'среднее'
data['Образование'] = data['Образование и ВУЗ'].apply(get_education)
data = data.drop('Образование и ВУЗ', axis=1)
print(data['Образование'].value_counts()['среднее'])

559


In [6]:
# Школа

def get_sex(arg):
    if 'Мужчина' in arg:
        return 'М'
    else:
        return 'Ж'
    
def get_age(arg):
    arg_splitted = arg.split(' ')
    year_words=['год', 'года', 'лет']
    for index, item in enumerate (arg_splitted):
        if item in year_words:
            return int(arg_splitted[index-1])

data['Пол'] = data['Пол, возраст'].apply(get_sex)
data['Возраст'] = data['Пол, возраст'].apply(get_age)
data = data.drop('Пол, возраст', axis=1)

print(round(data['Пол'].value_counts(normalize=True)['Ж'] * 100, 2))
print(round(data['Возраст'].mean(), 2))

19.07
32.2


In [7]:
# Опыт работы

def get_experience(arg):
    if arg is np.nan:
        return None
    year_words=['год', 'года', 'лет']
    month_words=['месяц', 'месяца', 'месяцев']
    arg_splitted = arg.split(' ')[:7]
    years = 0
    months = 0
    for index, item in enumerate (arg_splitted):
        if item in year_words:
            years = int(arg_splitted[index-1])
        if item in month_words:
            months = int(arg_splitted[index-1])
    return int(years*12 + months)
    
data['Опыт работы (месяц)'] = data['Опыт работы'].apply(get_experience)
data = data.drop('Опыт работы', axis=1)
print(round(data['Опыт работы (месяц)'].median()))

100


In [8]:
def get_city(arg):
    million_cities = ['Новосибирск', 'Екатеринбург', 'Нижний Новгород',
                      'Казань', 'Челябинск', 'Омск', 'Самара', 'Ростов-на-Дону', 
                      'Уфа', 'Красноярск', 'Пермь', 'Воронеж', 'Волгоград'
                     ]
    city = arg.split(' , ')[0]
    if (city == 'Москва') or (city == 'Санкт-Петербург'):
        return city
    elif city in million_cities:
        return 'город миллионник'
    else:
        return 'другие'
    
def get_ready_to_move(arg):
    if ('не готов к переезду' in arg) or ('не готова к переезду' in arg):
        return False
    else:
        return True
    
def get_ready_for_bisiness_trips(arg):
    if ('не готов к командировкам' in arg) or('не готова к командировкам' in arg):
        return False
    else: 
        return True
    
data['Город'] = data['Город, переезд, командировки'].apply(get_city)
data['Готовность к переезду'] = data['Город, переезд, командировки'].apply(get_ready_to_move)
data['Готовность к командировкам'] = data['Город, переезд, командировки'].apply(get_ready_for_bisiness_trips)
data = data.drop('Город, переезд, командировки', axis=1)
print(round(data['Город'].value_counts(normalize=True)['Санкт-Петербург'] * 100)) 
print(round(data[
    data['Готовность к переезду'] & data['Готовность к командировкам']
].shape[0] / data.shape[0] *100))

11
32


In [9]:
# занятость и график
employments = ['полная занятость', 'частичная занятость',
              'проектная работа', 'волонтерство', 'стажировка']
charts = ['полный день', 'сменный график', 
         'гибкий график', 'удаленная работа',
         'вахтовый метод']
for employment, chart in zip(employments, charts):
    data[employment] = data['Занятость'].apply(lambda x: employment in x)
    data[chart] = data['График'].apply(lambda x: chart in x)
data = data.drop('Занятость', axis=1)
data = data.drop('График', axis=1)
print(data[data['проектная работа'] & data['волонтерство']].shape[0])
print(data[data['вахтовый метод'] & data['гибкий график']].shape[0])

436
2311


In [10]:
# зарплата

def get_salary_num(arg):
    salary = float(arg.split(' ')[0])
    return salary

def get_salary_сurrency(arg):
    currency_dict = {
        'USD': 'USD', 'KZT': 'KZT',
        'грн': 'UAH', 'белруб': 'BYN',
        'EUR': 'EUR', 'KGS': 'KGS',
        'сум': 'UZS', 'AZN': 'AZN'
    }
    curr = arg.split(' ')[1].replace('.', '')
    if curr == 'руб':
        return 'RUB'
    else:
        return currency_dict[curr]
    
rates = pd.read_csv('ExchangeRates.csv')
rates['date'] = pd.to_datetime(rates['date']).dt.date
data['Обновление резюме'] = pd.to_datetime(data['Обновление резюме']).dt.date
data['ЗП (tmp)'] = data['ЗП'].apply(get_salary_num)
data['Курс (tmp)'] = data['ЗП'].apply(get_salary_сurrency)
merged = data.merge(
    rates, 
    left_on=['Курс (tmp)', 'Обновление резюме'],
    right_on=['currency', 'date',], 
    how='left'
)
merged['close'] = merged['close'].fillna(1)
merged['proportion'] = merged['proportion'].fillna(1)
data['ЗП (руб)'] = merged['close'] * merged['ЗП (tmp)'] / merged['proportion']
data = data.drop(['ЗП', 'ЗП (tmp)', 'Курс (tmp)'], axis=1)
print(round(data['ЗП (руб)'].median()/1000))

59


In [38]:
data.to_csv('hh_database_preprocessed.csv', index=False)

In [39]:
data2 = pd.read_csv('hh_database_preprocessed.csv')
data2.shape

(44744, 23)

In [40]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44744 entries, 0 to 44743
Data columns (total 23 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Ищет работу на должность:        44744 non-null  object 
 1   Последнее/нынешнее место работы  44743 non-null  object 
 2   Последняя/нынешняя должность     44742 non-null  object 
 3   Обновление резюме                44744 non-null  object 
 4   Авто                             44744 non-null  object 
 5   Образование                      44744 non-null  object 
 6   Пол                              44744 non-null  object 
 7   Возраст                          44744 non-null  int64  
 8   Опыт работы (месяц)              44576 non-null  float64
 9   Город                            44744 non-null  object 
 10  Готовность к переезду            44744 non-null  bool   
 11  Готовность к командировкам       44744 non-null  bool   
 12  полная занятость  

In [49]:
data2 = pd.read_csv('tyurin_hh_data_preprocessed.csv')

duplicates = data2[data2.duplicated(subset=data2.columns)]
data2 = data2.drop_duplicates()
print(duplicates.shape[0])

158


In [50]:
null_data = data2.isnull().sum()
display(null_data[null_data > 0])

Последнее/нынешнее место работы      1
Последняя/нынешняя должность         2
Опыт работы (месяц)                168
Готовность к командировкам          70
dtype: int64

In [43]:
# data = pd.read_csv('hh_database.csv', sep=';')

In [44]:
data2[data2['Опыт работы (месяц)'] == 0]

,Ищет работу на должность:,Последнее/нынешнее место работы,Последняя/нынешняя должность,Обновление резюме,Авто,Образование,Пол,Возраст,Опыт работы (месяц),Город,...,полный день,частичная занятость,сменный график,проектная работа,гибкий график,волонтерство,удаленная работа,стажировка,вахтовый метод,ЗП (руб)
19199,системный администратор / инженер программист,"АО ""Предприятие В-1336""",Инженер-программист,2019-10-04,Не указано,высшее,М,39,0.0,город миллионник,...,True,False,False,False,False,False,False,False,False,45000.0
26781,Системный администратор,"Европа, Торговая сеть",Системный администратор,2019-10-04,Имеется собственный автомобиль,высшее,М,25,0.0,другие,...,True,False,True,False,True,False,True,False,True,30000.0


In [47]:
data2 = data2[data2['Опыт работы (месяц)'] != 0]

In [48]:
data2 = data2.dropna(subset=['Последнее/нынешнее место работы', 'Последняя/нынешняя должность'])
data2['Опыт работы (месяц)'] = data2['Опыт работы (месяц)'].fillna(data2['Опыт работы (месяц)'].median())
print(round(data2['Опыт работы (месяц)'].mean(), 2))

114.36


In [53]:
import pandas as pd 

df1 = pd.read_csv('hh_database_preprocessed.csv')

df2 = pd.read_csv('tyurin_hh_data_preprocessed.csv')

# df1.compare(df2)

In [56]:
df3 = df1.compare(df2, align_axis=0)

In [58]:
df3.head()

Обновление резюме  Возраст Город Готовность к командировкам  ЗП (руб)
0 self         2019-04-16      NaN   NaN                        NaN       NaN
  other  16.04.2019 15:59      NaN   NaN                        NaN       NaN
1 self         2019-12-04      NaN   NaN                        NaN       NaN
  other  12.04.2019 08:42      NaN   NaN                        NaN       NaN
2 self         2019-04-16      NaN   NaN                        NaN       NaN

In [57]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 89488 entries, (0, 'self') to (44743, 'other')
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Обновление резюме           89488 non-null  object 
 1   Возраст                     2 non-null      float64
 2   Город                       14664 non-null  object 
 3   Готовность к командировкам  72 non-null     object 
 4   ЗП (руб)                    488 non-null    float64
dtypes: float64(2), object(3)
memory usage: 4.2+ MB
